In [3]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import plotly.subplots as sp
from sympy import symbols, solve
import matplotlib.pyplot as plt


## Import data

In [ ]:
df_data = pd.read_excel('data_with_mean_ESG.xlsx', header=[0,1], index_col=0)

## Filtering stocks 

In [ ]:
def filter_esg_stocks(df, score_type, threshold=None):
    valid_score_type = ['ESG', 'Enviromental', 'Social', 'Governance']
    if score_type not in valid_score_type:
        raise ValueError("Invalid score type. Please choose from ESG, Enviromental, Social or Governance")

    # Filter dataframe based on threshol
    if threshold is not None:
        filtered = df[df[score_type + ' Disc Mean'][score_type + ' Disc mean score'] >= threshold]

    return filtered

In [ ]:
filtered = filter_esg_stocks(df_data, 'ESG', threshold=40)
filtered

## Creating a portfolio

In [ ]:
def create_portfolio1(dataframe):
    """
    Function to interactively create a portfolio of stocks from a DataFrame.

    Args:
        dataframe (pd.DataFrame): DataFrame containing stock tickers, last prices, and ESG scores.

    Returns:
        pd.DataFrame: DataFrame containing selected stocks with their respective variables and scores.
    """

    #Creating a copy of the input stock Dataframe to avoid modifying orginal data
    stock_data_copy = dataframe.copy()


    #Creating an empty Dataframe to store selected stocks in
    portfolio = pd.DataFrame(columns=dataframe.columns)

    ticker_list = list(dataframe.index)


    # Create widgets for user input
    num_stocks_widget = widgets.IntSlider(
        value=1,
        min=1,
        max=len(dataframe),
        step=1,
        description='Number of stocks:',
        continuous_update=False
    )
    stock_dropdown_widget = widgets.Dropdown(
        options=ticker_list,
        description='Stocks:',
        continuous_update=False,
        layout=widgets.Layout(width='300px')
    )
    add_button = widgets.Button(description='Add stock')
    exit_button = widgets.Button(description='Exit')
    output = widgets.Output()
    
    # Define button click event handlers
    def add_stock(button):
        nonlocal stock_data_copy, portfolio

        #Here we get the selected ticker from the dropdown menu
        stock_ticker = stock_dropdown_widget.value

        #Now we need to check whether the selected stock is in the portfolio
        if stock_ticker in portfolio.index:
            #If the stock is in the portfolio the user will recieve an message
            print(f"Stock {stock_ticker} is already placed in your portfolio. Please choose another stock")
        else:
            #If the stock isn't already present in the portfolio it will be added to the users portfolio.
            stock_data = dataframe.loc[stock_ticker]
            portfolio.loc[stock_ticker] = stock_data
            print(f"Stock {stock_ticker} has been added to your portfolio")

        #Lastly, we need to remove the added stock from the dropdown menu
        ticker_list.remove(stock_ticker)
        stock_dropdown_widget.options = ticker_list
        
            
            
            # Adding the selected stock to the users portfolio 
            #stock_ticker = stock_data_copy[stock_data_copy.loc[stock_ticker] == stock_ticker]
            #portfolio = pd.concat([portfolio, stock_ticker])

            #Now we need to remove the stock from the dropdown menu
            #stock_data_copy = stock_data_copy[stock_data_copy.loc[stock_ticker] != stock_ticker]
            #stock_dropdown_widget.options = dataframe.index

            #with output:
             #   print(f"Added stock to portfolio: {stock_ticker}")
              #  display(stock_ticker)

       #else:
        #    with output:
         #       print(f"Stock {stock_ticker} is already in your portfolio")




        #stock_data = dataframe.loc[stock_ticker]
        #portfolio.loc[stock_ticker] = stock_data
        #print(f'Stock {stock_ticker} added to portfolio.')

    def exit(button):
        display(portfolio)
        return


    #Creating a reset button, from which the user can clear the portfolio
    reset_button = widgets.Button(description='Reset portfolio')
    reset_output = widgets.Output()


    def reset_portfolio(button):
        nonlocal stock_data_copy, portfolio

        #Clearing the portfolio and reseting the dropdown menu options
        portfolio = pd.DataFrame(columns=dataframe.columns)
        stock_data_copy = dataframe.copy()
        stock_dropdown_widget.options = dataframe.index

        #Clear the output
        with reset_output:
            print("Your portfolio has been reset.")
            display(pd.DataFrame())

        #Adding the reset button
        reset_button.on_click(reset_portfolio)



    # Assign event handlers to buttons
    add_button.on_click(add_stock)
    exit_button.on_click(exit)

    # Display widgets
    display(num_stocks_widget)
    display(stock_dropdown_widget)
    display(add_button)
    #display(output)
    display(exit_button)
    display(reset_button)
    display(reset_output)

    return portfolio

In [ ]:
selected = create_portfolio1(filtered)

## Mean ESG scores

In [ ]:
esg = selected['ESG Disc Mean']['ESG Disc mean score']
esg_np = esg.to_numpy()
e = selected['Enviromental Disc Mean']['Enviromental Disc mean score']
e_np = e.to_numpy()
s = selected['Social Disc Mean']['Social Disc mean score']
s_np = s.to_numpy()
g = selected['Governance Disc Mean']['Governance Disc mean score']
g_np = g.to_numpy()

In [ ]:
esg_np = esg_np / 100

## Log returns

### For traditional investor

In [ ]:
from log_return import log_returns
log_re = log_returns(selected['Last Price']).replace([np.inf, -np.inf], np.nan).dropna(axis=1).T

### For ESG motivated investor

In [ ]:
log_re_esg = log_re*esg_np

### Labels

In [ ]:
labels = log_re.columns
np_labels = labels.tolist()
np_labels

## Rolling cov and return

### For traditional investor

In [ ]:
from rolling_cov_re import rolling_er_cov
ret, cov, inv_cov = rolling_er_cov(log_re, 5)


### For ESG motivated investor

In [ ]:
ret_esg, cov_esg, inv_cov_esg = rolling_er_cov(log_re_esg, 5)


### VECTOR OF ONES

In [ ]:
from ones import vector_ones
ones_vector = vector_ones(np_labels)


In [ ]:
ones_vector = ones_vector.tolist()

## Calculate MVP

### useful equations are calculated for us to derive the minimum portfolio

In [ ]:
def mvp_func_rolling(inv_covs, mean_returns, vec_ones):
    list_a = []
    list_b = []
    list_c = []
    list_d = []
    for i in range(len(inv_covs)):
        inv_cov = inv_covs[i]
        mean_return = mean_returns[i]

        a = np.array(np.dot(np.dot(mean_return.T, inv_cov), vec_ones))
        b = np.array(np.dot(np.dot(mean_return.T, inv_cov), mean_return))
        c = np.array(np.dot(np.dot(vec_ones, inv_cov), vec_ones))
        d = np.array(b*c - a**2)
        list_a.append(a)
        list_b.append(b)
        list_c.append(c)
        list_d.append(d)
    return (np.array(list_a), np.array(list_b), np.array(list_c), np.array(list_d))

### For traditional investor

In [ ]:
a, b, c, d = mvp_func_rolling(inv_cov, ret, ones_vector)

### For ESG motivated investor

In [ ]:
a_esg, b_esg, c_esg, d_esg = mvp_func_rolling(inv_cov_esg, ret_esg, ones_vector)

## Finding the optimal weights for minimal variance portfolio, with two different expected returns

### For traditional investor

In [ ]:
from weights_mvp import mvp_weights
w1_list = []
w2_list = []
for i in range(len(ret)):
    w1, w2 = mvp_weights(a[i], b[i], c[i], d[i], inv_cov[i], ones_vector, ret[i], 0.22, 0.28)
    w1_list.append(w1)
    w2_list.append(w2)
w1_list = np.array(w1_list)
w2_list = np.array(w2_list)


In [ ]:
w1_list

### For ESG motivated investor

In [ ]:
w1_esg_list = []
w2_esg_list = []
for i in range(len(ret)):
    w1_esg, w2_esg = mvp_weights(a_esg[i], b_esg[i], c_esg[i], d_esg[i], inv_cov_esg[i], ones_vector, ret_esg[i], 0.22, 0.28)
    w1_esg_list.append(w1_esg)
    w2_esg_list.append(w2_esg)
w1_esg_list = np.array(w1_esg_list)
w2_esg_list = np.array(w2_esg_list)


In [ ]:
w1_esg_list

### Creating a list that gives values between any given value inside a list

In [ ]:
from alpha import alpha
alpha_list = alpha([-1])

## We can now use the two fund theorem to find the weights for the entire effecient frontier, using two optimal solutions

### For traditional investor

In [ ]:
from two_fun_theorem import w3
ef_weights_list = []
for i in range(len(ret)):
    ef_weights = w3(w1_list[i], w2_list[i], alpha_list)
    ef_weights_list.append(ef_weights)
ef_weights_list = np.array(ef_weights_list)

In [ ]:
ef_weights_list

### For ESG motivated investor

In [ ]:
ef_esg_weights_list = []
for i in range(len(ret)):
    ef_esg_weights = w3(w1_esg_list[i], w2_esg_list[i], alpha_list)
    ef_esg_weights_list.append(ef_esg_weights)
ef_esg_weights_list = np.array(ef_esg_weights_list)



In [ ]:
sum(ef_esg_weights_list[0][0])

## Labels

In [ ]:
tickers_list = []
for i in range(len(ef_weights_list[0])):
        #assets = np.random.choice(list(labels), len(labels), replace=False)
        assets = list(labels)
        tickers_list.append(assets)

### Filtering through the weights. This portion of the code will remove small weights and redistribute their weight to the rest of the weights

In [ ]:
#Define the threshold percentages for removing small weights and the allowed weight range for removing portfolios:
threshold_percentage = 1  # remove weights with less than 1% contribution
allowed_range = (-0.1, 1)  # remove portfolios with weights between -10% and 10%

In [ ]:
filtered_weights = []
filtered_tickers = []

for i in range(len(ef_weights_list)):
    for j in range(len(ef_weights_list[0])):
        # get the weights for the current portfolio
        weights = ef_weights_list[i][j]
        
        # remove small weights and calculate the total weight of remaining assets
        total_weight = sum(weights)
        percentage_contributions = [weight/total_weight for weight in weights]
        small_weights = [idx for idx, percentage in enumerate(percentage_contributions) if percentage < threshold_percentage/100]
        filtered_weights_inner = [weight for idx, weight in enumerate(weights) if idx not in small_weights]
        filtered_tickers_inner = [ticker for idx, ticker in enumerate(tickers_list[j]) if idx not in small_weights]
        
        # remove portfolios with weights between -10% and 10%
        if any(abs(weight) >= abs(allowed_range[0]) and abs(weight) <= allowed_range[1] for weight in filtered_weights_inner):
            continue
        
        # distribute the weight of the removed assets proportionally among the remaining assets
        removed_weight = sum(weights[idx] for idx in small_weights)
        remaining_weight = sum(filtered_weights_inner)
        if removed_weight > 0:
            proportion = remaining_weight / (remaining_weight + removed_weight)
            added_weight = removed_weight * proportion
            filtered_weights_inner = [weight + added_weight * weight/remaining_weight if weight not in small_weights else weight for weight in filtered_weights_inner]
        
        # add the filtered weights to the list of filtered weights
        filtered_weights.append(filtered_weights_inner)
        
        # add the corresponding ticker to the list of filtered tickers
        filtered_tickers.append(filtered_tickers_inner)

In [ ]:
filtered_weights

In [ ]:
# calculate the sum of the filtered weights
sum_filtered_weights = [sum(weights) for weights in filtered_weights]

# normalize the filtered weights to ensure that they sum to 1
normalized_filtered_weights = [[weight/sum_weight for weight in weights] for weights, sum_weight in zip(filtered_weights, sum_filtered_weights)]

## Calculating the expected return for each portfolio on the effecient frontier

### For traditional investor

In [ ]:
sum(ef_esg_weights_list[2][1] * ret_esg[2])

In [ ]:
list_return = []
# create a list of empty lists for each rolling window
rolling_returns = [[] for _ in range(len(ef_weights_list))]  
for k in range(len(ef_weights_list)):
    for j in range(len(ef_weights_list[0])):
        rolling_er = 0
        for x in range(len(np_labels)):
            rolling_er += ef_weights_list[k][j][x] * ret[k][x]
            #list_return.append(rolling_er)
        # append the calculated return to the appropriate rolling window's list
        rolling_returns[k].append(rolling_er) 
rolling_returns = np.array(rolling_returns)

In [ ]:
return_list = []

In [ ]:
rolling_returns[1]

### For ESG motivated investor

In [ ]:
list_esg_return = []
# create a list of empty lists for each rolling window
rolling_returns_esg = [[] for _ in range(len(ef_esg_weights_list))]  
for k in range(len(ef_esg_weights_list)):
    for j in range(len(ef_esg_weights_list[0])):
        rolling_esg_er = 0
        for x in range(len(np_labels)):
            rolling_esg_er += ef_esg_weights_list[k][j][x] * ret_esg[k][x]
        list_esg_return.append(rolling_esg_er)
        # append the calculated return to the appropriate rolling window's list
        rolling_returns_esg[k].append(rolling_esg_er) 
rolling_returns_esg = np.array(rolling_returns_esg)

In [ ]:
rolling_returns_esg[1]

### For traditional investor

In [ ]:
from variance import var_portfolio
var_list = []
for i in range(len(ret)):
    port_variance = var_portfolio(ef_weights_list[i], cov[i])
    var_list.append(port_variance)

In [ ]:
from volatility import volatility
vol_list = []
for i in range(len(ret)):
    port_vol = volatility(var_list[i])
    vol_list.append(port_vol)
vol_list = np.array(vol_list)

In [ ]:
len(vol_list[0])

### For ESG motivated investor

In [ ]:
vol_list

In [ ]:
var_esg_list = []
for i in range(len(ret)):
    port_esg_variance = var_portfolio(ef_esg_weights_list[i], cov_esg[i])
    var_esg_list.append(port_esg_variance)

In [ ]:
vol_esg_list = []
for i in range(len(ret)):
    port_esg_vol = volatility(var_esg_list[i])
    vol_esg_list.append(port_esg_vol)
vol_esg_list = np.array(vol_esg_list)

In [ ]:
vol_esg_list

## Weighted average ESG score for the efficient fronter

In [ ]:
from weighted_average import aw_esg
assets = list(labels)

wa_esg_list = []
wa_e_list = []
wa_s_list = []
wa_g_list = []
for i in range(len(ef_weights_list)):
    #For the whole efficient frontier
    weighted_av_esg = aw_esg(esg_np, ef_weights_list[i], assets)
    wa_esg_list.append(weighted_av_esg)
    weighted_av_e = aw_esg(e_np, ef_weights_list[i], assets)
    wa_e_list.append(weighted_av_e)
    weighted_av_s = aw_esg(s_np, ef_weights_list[i], assets)
    wa_s_list.append(weighted_av_s)
    weighted_av_g = aw_esg(g_np, ef_weights_list[i], assets)
    wa_g_list.append(weighted_av_g)

## Calculating sharpe ratio point

### Finding the optimal weights for each rolling window

In [ ]:
sr_w_list = []
for i in range(len(rolling_returns)):
    # Calculate sharpe ratio for current rolling window
    sr = (rolling_returns[i] - 0.01) / var_list[i]
    # Find index of max sharpe ratio for current window
    max_sr_index = sr.argmax()
    # Use index to get corresponding weights from efficient frontier
    sr_weights = ef_weights_list[i][max_sr_index]
    sr_w_list.append(sr_weights)

### Calculating the expected return and volatility for each of the optimal weights

#### Expected return

In [ ]:
#r_sr = exp_return(weights_sr, mean_re)
er_sr_list = []
for i in range(len(sr_w_list)):
    sr_r = sr_w_list[i]@ret[i]
    er_sr_list.append(sr_r)

### Volatility

In [ ]:
vol_sr_list = []
for i in range(len(sr_w_list)):
    vol_sr = np.sqrt(sr_w_list[i]@cov[i]@sr_w_list[i])
    vol_sr_list.append(vol_sr)

## Finding the sharpe ratio for each sharpe point 

In [ ]:
sr_list = []
for i in range(len(er_sr_list)):
    sharpe_ratio = (er_sr_list[i] - 0.01) / var_list[i]
    sr_list.append(sharpe_ratio)

### ESG score weighted average

In [ ]:
list_esg_wa = []
for i in range(len(ef_esg_weights_list)):
    esg_wa_1 = np.dot(ef_esg_weights_list[i], esg_np)
    list_esg_wa.append(esg_wa_1)

## ESG Sharpe Ratio Point

In [ ]:
sr_w_esg = []
for i in range(len(rolling_returns_esg)):
    # Calculate sharpe ratio for current rolling window
    sr_esg = ((rolling_returns_esg[i] - 0.01) / var_list[i]) + 2*list_esg_wa[i]
    # Find index of max sharpe ratio for current window
    max_sr_esg_index = sr_esg.argmax()
    # Use index to get corresponding weights from efficient frontier
    sr_esg_weights = ef_esg_weights_list[i][max_sr_esg_index]
    sr_w_esg.append(sr_esg_weights)

In [ ]:
sr_w_esg

### Expected return and volatility of ESG Sharpe ratio point

### Expected return

In [ ]:
#r_sr = exp_return(weights_sr, mean_re)
er_sr_esg_list = []
for i in range(len(sr_w_esg)):
    sr__esg_r = sr_w_esg[i]@ret_esg[i]
    er_sr_esg_list.append(sr__esg_r)

### Volatility

In [ ]:
vol_sr_esg_list = []
for i in range(len(sr_w_esg)):
    vol_esg_sr = np.sqrt(sr_w_esg[i]@cov[i]@sr_w_esg[i])
    vol_sr_esg_list.append(vol_esg_sr)

### Tickers for SR

In [ ]:
sr_t_list = []
for i in range(len(sr_w_list)):
    assets = list(labels)
    sr_t_list.append(assets)

## Calculating MVP

### Finding the weights of MVP in each rolling window

In [ ]:
mvp_w_list = []
for i in range(len(ef_weights_list)):
    min_vol_index = vol_list[i].argmin()
    
    weights_mvp = ef_weights_list[i][min_vol_index]
    mvp_w_list.append(weights_mvp)

### Tickers for MVP

In [ ]:
mvp_t_list = []
for i in range(len(mvp_w_list)):
    assets = list(labels)
    mvp_t_list.append(assets)

### Calculating the expected return and volatility for MVP in each rolling window

#### Expected return

In [ ]:
er_mvp_list = []
for i in range(len(mvp_w_list)):
    mvp_r = mvp_w_list[i]@ret[i]
    er_mvp_list.append(mvp_r)

### Variance and volatility

In [ ]:
vol_mvp_list = []
for i in range(len(mvp_w_list)):
    vol_mvp = np.sqrt(mvp_w_list[i]@cov[i]@mvp_w_list[i])
    vol_mvp_list.append(vol_mvp)

## Weighted average ESG score For the SR point and MVP

In [ ]:
#Hvis man investerer (long) i aktier med høj score og short sælger aktier med en lav score, så forstærker det de høje scores. 

In [ ]:
def aw_score(mean_esg_scores, weight):
    weighted_avg_esg_scores = []
    for window_weights in weight:
        # Multiply weights with ESG scores
        weighted_esg_scores = [weight * mean_esg_scores[i] for i, weight in enumerate(window_weights)]
        # Sum up weighted ESG scores
        sum_weighted_esg = sum(weighted_esg_scores)
        # Sum up weights
        sum_weights = sum(window_weights)
        # Calculate weighted average ESG score
        weighted_avg_esg = sum_weighted_esg / sum_weights
        # Add to result list
        weighted_avg_esg_scores.append(weighted_avg_esg)

    return weighted_avg_esg_scores

### For SR point

In [ ]:
sr_esg_list = aw_score(esg_np, sr_w_list)
sr_e_list = aw_score(e_np, sr_w_list)
sr_s_list = aw_score(s_np, sr_w_list)
sr_g_list = aw_score(g_np, sr_w_list)

### For MVP 

In [ ]:
mvp_esg_list = aw_score(esg_np, mvp_w_list)
mvp_e_list = aw_score(e_np, mvp_w_list)
mvp_s_list = aw_score(s_np, mvp_w_list)
mvp_g_list = aw_score(g_np, mvp_w_list)


# Efficient frontier plot for each rolling window

In [ ]:
color_scale = [
    [0, 'red'],  # Low ESG score -> red
    [0.5, 'yellow'],  # Mid ESG score -> yellow
    [1, 'green']  # High ESG score -> green
]

for i in range(len(rolling_returns)):


    weights_str_sr = [f'{w*100:.2f}%' for w in sr_w_list[i]]
    weights_text_sr = ', '.join(weights_str_sr)

    #filter out returns and volatility less than or equal to -0.5
    filtered_returns = []
    filtered_volatility = []
    filtered_weights = []
    filtered_wa_esg = []
    filtered_wa_e = []
    filtered_wa_s = []
    filtered_wa_g = []
    filtered_esg_sr = []
    filtered_esg_returns = []
    filtered_esg_weights = []
    for j in range(len(rolling_returns[i])):
        #if rolling_returns[i][j] > -0.3:
            filtered_returns.append(rolling_returns[i][j])
            filtered_volatility.append(vol_list[i][j])
            filtered_weights.append(ef_weights_list[i][j])
            filtered_wa_esg.append(wa_esg_list[i][j])
            filtered_wa_e.append(wa_e_list[i][j])
            filtered_wa_s.append(wa_s_list[i][j])
            filtered_wa_g.append(wa_g_list[i][j])
            filtered_esg_returns.append(rolling_returns_esg[i][j])
            filtered_esg_weights.append(ef_esg_weights_list[i][j])

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=filtered_volatility, 
        y=filtered_returns, 
        mode ='markers',
        name='Efficient Frontier',
        marker=dict(
        color=filtered_wa_esg,
        colorscale=color_scale,
        colorbar=dict(title='WA ESG Score')
        ),
        text=[f'Weights: {", ".join([f"(Ticker: {ticker}, Weight: {w*100:.2f}%)" for ticker, w in zip(tickers, wts)])}<br>' +
        f'Volatility: {vol:.2%}<br>' +
        f'Return: {rets:.2%}<br>' +
        f'WA ESG Score: {esg:.4}<br>' +
        f'WA Enviromental Score: {e:.4}<br>' +
        f'WA Social Score: {s:.4}<br>' +
        f'WA Governance Score: {g:.4}'
        for tickers, wts, vol, rets, esg, e, s, g in zip(tickers_list*len(filtered_weights), filtered_weights, filtered_volatility, filtered_returns, filtered_wa_esg, filtered_wa_e, filtered_wa_s, filtered_wa_g)],
        hovertemplate='%{text}',
        hoverlabel=dict(
            font_size=10)
        ))

    fig.add_trace(go.Scatter(
        x=filtered_volatility, 
        y=filtered_esg_returns, 
        mode ='markers',
        name='ESG Efficient Frontier',
        text=[f'Weights: {", ".join([f"(Ticker: {ticker}, Weight: {w*100:.2f}%)" for ticker, w in zip(tickers, wts)])}<br>' +
        f'Volatility: {vol:.2%}<br>' +
        f'Return: {rets:.2%}'
        for tickers, wts, vol, rets in zip(tickers_list*len(filtered_esg_weights), filtered_esg_weights, filtered_volatility, filtered_esg_returns)],
        hovertemplate='%{text}',
        hoverlabel=dict(
            font_size=10)
        ))


    fig.add_trace(go.Scatter(x = [vol_sr_list[i]], y = [er_sr_list[i]],
                        marker=dict(color='red', size=12), 
                        mode='markers',
                        name='SR portfolio',
                        text=[f'Weights: {", ".join([f"(Ticker: {t_sr}, Weight: {w_sr*100:.2f}%)" for t_sr, w_sr in zip(tickers_sr, wts_sr)])}<br>' +
                        f'Volatility: {vol_sr:.2%}<br>' +
                        f'Return: {rets_sr:.2%} <br>' +
                        f'Sharpe ratio: {sharpe}<br>' +
                        f'WA ESG Score: {esg_sr:.4}'
                        for tickers_sr, wts_sr, vol_sr, sharpe, rets_sr, esg_sr in zip(sr_t_list*len(sr_w_list), sr_w_list, vol_sr_list, sr_list, er_sr_list, sr_esg_list)],
                        hovertemplate='%{text}',
                        hoverlabel=dict(
                        font_size=10)
        ))


    fig.add_trace(go.Scatter(x = [vol_sr_list[i]], y = [er_sr_esg_list[i]],
                        marker=dict(color='blue', size=12), 
                        mode='markers',
                        name=' ESG SR portfolio',
                        text=[f'Weights: {", ".join([f"(Ticker: {t_sr}, Weight: {w_sr*100:.2f}%)" for t_sr, w_sr in zip(tickers_sr, wts_sr)])}<br>' +
                        f'Volatility: {vol_sr:.2%}<br>' +
                        f'Return: {rets_sr:.2%}'
                        #f'Sharpe ratio: {sharpe}<br>' +
                        #f'WA ESG Score: {esg_sr:.4}'
                        for tickers_sr, wts_sr, sharpe, rets_sr in zip(sr_t_list*len(sr_w_esg), sr_w_esg, vol_sr_esg_list, er_sr_esg_list)],
                        hovertemplate='%{text}',
                        hoverlabel=dict(
                        font_size=10)
        ))

    fig.add_trace(go.Scatter(x = [vol_mvp_list[i]], y = [er_mvp_list[i]],
                        marker=dict(color='White', size=12), 
                        mode='markers',
                        name='Minimum Variance Portfolio',
                        text=[f'Weights: {", ".join([f"(Ticker: {t_mvp}, Weight: {w_mvp*100:.2f}%)" for t_mvp, w_mvp in zip(tickers_mvp, wts_mvp)])}<br>' +
                        f'Volatility: {vol_mvp:.2%}<br>' +
                        f'Return: {rets_mvp:.2%} <br>'
                        for tickers_mvp, wts_mvp, vol_mvp, rets_mvp in zip(mvp_t_list*len(mvp_w_list), mvp_w_list, vol_mvp_list, er_mvp_list)],
                        hovertemplate='%{text}',
                        hoverlabel=dict(
                        font_size=10)
        ))




    fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1    
    ))


    # Update layout
    fig.update_layout(
        template='plotly_dark',
        title='Efficient Frontier',
        xaxis_title='Volatility',
        yaxis_title='Return',
        hovermode='closest',
        width=1000,
        height=600,
        margin=dict(l=50, r=50, b=50, t=50, pad=4)
    )

    fig.show()

# Backtest

In [ ]:
#sr_w_list
#cov
#ret

In [ ]:
def ex_post(opt_weightlist, returns, cov):
    realized_vol = []
    realized_return = []
    returns_np = np.array(returns)
    
    for i in range(len(opt_weightlist)):
        weight = np.array(opt_weightlist[i])
        # Calculate realized return:
        realized_return.append(weight @ returns_np[i])

        # Calculate realized volatility
        realized_vol.append(np.sqrt(weight @ cov[i] @ weight))

    return realized_return, realized_vol

In [ ]:
ex_post_return, ex_post_std = ex_post(sr_w_list, ret, cov)

In [ ]:
ex_post_return

# Asset allocation

In [ ]:
#Helper function to find the amount that should be invested in the risk free asset and the risky asset
#PAGE 82 IN DATFIN TEXTBOOK
def tobin_solve(rf,sr_return, sr_vol,target):
    tobin_weights = []
    tobin_vol = []
    for i in range(len(sr_vol)):
        exp = sr_return[i]
        vol = sr_vol[i]
        y = symbols('y')
        expr = rf + ( ( exp - rf ) / vol ) * y - target

        vol_c = solve(expr)
        
        x = vol_c / vol
        
        tobin_exp = (1-x) * rf + x * exp
        
        print("Amount invested in risky asset=",x[0])
        print("Expected return =",tobin_exp[0])
        print("Expected vol=",vol_c[0])
        print("\n")
        
        tobin_vol.append(vol_c)
        tobin_weights.append(x[0])
    return tobin_weights, tobin_vol

In [ ]:
tobin_weights, tobin_vol = tobin_solve(0.01, er_sr_list, vol_sr_list,0.10)

In [ ]:
tobin_weights = np.array(tobin_weights)

In [ ]:
#
# Helper function to find the turnover change
def turnover(tobin_weights,opt_weightlist):
    
    x_turnover = []
    weights_turnover = []
    #opt_scaled = []
    
    for i in range(len(tobin_weights)-1):
        #opt_scaled.append(tobin_weights[i]*opt_weightlist[i])
        #opt_scaled.append(tobin_weights[i+1]*opt_weightlist[i+1])
        
        x_turnover.append(abs(tobin_weights[i]-tobin_weights[i+1]))
        
        weights_turnover.append(np.sum(np.abs((tobin_weights[i]*opt_weightlist[i])-(tobin_weights[i+1]*opt_weightlist[i+1]))))
    
    return x_turnover, weights_turnover

In [ ]:
x_turnover, weights_turnover = turnover(tobin_weights, sr_w_list)

In [ ]:
#THE Y-AXIS IS THE PERCENTAGE OF HOW MUCH WE INVEST IN THE RISKLESS ASSET 
#Plot for turnover in amount invsted in risky/riskless asset
fig = plt.figure()
x = np.linspace(0,9,9)
plt.bar(x,x_turnover)
plt.grid()
plt.savefig("tobinturnover.png")
plt.show()

In [ ]:
#THE Y-AXIS IS THE PERCENTAGE CHANGE OF WEIGHT ALLOCATION IN OUR PORTFOLIO - THUS THE RISKY ASSETS
#Plot for turnover within weights in the portfolio
fig = plt.figure()
x = np.linspace(0,9,9)
plt.bar(x,weights_turnover)
plt.grid()
plt.savefig("weightsturnover.png")
plt.show()

# Getting Beta coefficients for each rolling window